In [6]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from string import punctuation

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amithasanshuvo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
tweet_df = pd.read_csv("train.csv", encoding="latin")


In [3]:
tweet_df.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [4]:
ps = PorterStemmer()
negation_list = ["arent","isnt","dont","doesnt","not","cant","couldnt", "werent",
                 "wont","didnt","never","nothing","nowhere","noone","none"
                "hasnt","hadnt","shouldnt","wouldnt","aint"]
               
def preProcessTweets(tweet):
    tweet = tweet.lower()
    tweet = re.sub('n[^A-Za-z ]t','nt', tweet)
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet)
    tweet = re.sub('@[^\s]+', '', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = word_tokenize(tweet)
    tweet_list = [];
    negate = False
    for word in tweet:
        word = ps.stem(word)
        if word in negation_list:
            negate = True
        elif negate is True and word in list(punctuation):
            negate = False
            
        if negate and word not in negation_list:
            word = "not_"+word
        else:
            pass
        word = re.sub('[^A-Za-z_ ]+', '', word)
        if len(word) > 2 and word not in stopwords.words('english'):
            tweet_list.append(word)
    tweet_set = set(tweet_list)
    return " ".join(tweet_set)

In [9]:
tweet_df["clean_text"] = tweet_df["SentimentText"].apply(preProcessTweets)


In [10]:
tweet_df.to_csv("train_preprocessed.csv")


In [11]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
import pickle

In [12]:
tweet_df_pre = pd.read_csv("train_preprocessed.csv")


In [13]:
tweet_df_pre.head()

,Unnamed: 0,ItemID,Sentiment,SentimentText,clean_text
0,0,1,0,is so sad for my APL frie...,friend sad apl
1,1,2,0,I missed the New Moon trail...,new miss moon trailer
2,2,3,1,omg its already 7:30 :O,alreadi omg
3,3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...,sinc thi sooo cri omgaga gunna dentist supos c...
4,4,5,0,i think mi bf is cheating on me!!! ...,t_t think cheat


In [14]:
tweet_df_pre["clean_text"]=tweet_df_pre["clean_text"].astype('U')


In [15]:
pipeline = Pipeline([
   ( 'bow',CountVectorizer()),
    ('classifier',MultinomialNB()),
])

In [17]:
from sklearn.model_selection import train_test_split
msg_train,msg_test,label_train,label_test = train_test_split(tweet_df_pre['clean_text'],tweet_df_pre['Sentiment'],test_size=0.2)

pipeline.fit(msg_train, label_train)

predictions = pipeline.predict(msg_test)

print(classification_report(predictions, label_test))

              precision    recall  f1-score   support

           0       0.63      0.73      0.68      7529
           1       0.82      0.74      0.78     12469

    accuracy                           0.74     19998
   macro avg       0.73      0.74      0.73     19998
weighted avg       0.75      0.74      0.74     19998



In [33]:

text ='you are bad'
pipeline.predict([text])[0]

0